<a href="https://colab.research.google.com/github/jamesl3483/aicalhacks_2023_team_sib/blob/main/hands_on_with_hume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hume + LLMs Workshop

This notebook uses the Hume batch API to analyze text and then convert emotion embeddings into text that can be used to condition an LLM like ChatGPT.

In [ ]:
!pip install hume
!pip install numpy

In [ ]:
from hume import HumeBatchClient
from hume.models.config import LanguageConfig

### Paste your API keys here

In [ ]:
HUME_API_KEY = "DpHWXrvooSKqQ2jCX6jqP4UoLOmlzpJc3q98AfpLiBANmYPA"

### Write some text with emotion to be processed

In [ ]:
from pathlib import Path

TEXT = "I'm so excited for the camping trip next weekend!"

filepath = "text.txt"
with open(filepath, "w") as fp:
    fp.write(TEXT)

### Submit a batch job

In [ ]:
client = HumeBatchClient(HUME_API_KEY)
config = LanguageConfig(granularity="sentence", identify_speakers=True)
job = client.submit_job(None, [config], files=[filepath])

print("Running...", job)

job.await_complete()
print("Job completed with status: ", job.get_status())

### Print out predictions

In [ ]:
from typing import Any, Dict, List


def print_emotions(emotions: List[Dict[str, Any]]) -> None:
    emotion_map = {e["name"]: e["score"] for e in emotions}
    for emotion in ["Excitement", "Joy", "Sadness", "Anger", "Confusion", "Fear"]:
        print(f"- {emotion}: {emotion_map[emotion]:4f}")

In [ ]:
emotion_embeddings = []
full_predictions = job.get_predictions()
for source in full_predictions:
    predictions = source["results"]["predictions"]
    for prediction in predictions:
        language_predictions = prediction["models"]["language"]["grouped_predictions"]
        for language_prediction in language_predictions:
            for chunk in language_prediction["predictions"]:
                print(chunk["text"])
                print_emotions(chunk["emotions"])
                emotion_embeddings.append(chunk["emotions"])
                print()

### Embedding-String Transformation

In [ ]:
import numpy as np
from typing import List


class Stringifier:
    RANGES = [(0.26, 0.35), (0.35, 0.44), (0.44, 0.53), (0.53, 0.62), (0.62, 0.71), (0.71, 10)]
    ADVERBS = ["slightly", "somewhat", "moderately", "quite", "very", "extremely"]

    ADJECTIVES_48 = [
        "admiring", "adoring", "appreciative", "amused", "angry", "anxious", "awestruck", "uncomfortable", "bored",
        "calm", "focused", "contemplative", "confused", "contemptuous", "content", "hungry", "determined",
        "disappointed", "disgusted", "distressed", "doubtful", "euphoric", "embarrassed", "disturbed", "entranced",
        "envious", "excited", "fearful", "guilty", "horrified", "interested", "happy", "enamored", "nostalgic",
        "pained", "proud", "inspired", "relieved", "smitten", "sad", "satisfied", "desirous", "ashamed",
        "negatively surprised", "positively surprised", "sympathetic", "tired", "triumphant"
    ]

    ADJECTIVES_53 = [
        "admiring", "adoring", "appreciative", "amused", "angry", "annoyed", "anxious", "awestruck", "uncomfortable",
        "bored", "calm", "focused", "contemplative", "confused", "contemptuous", "content", "hungry", "desirous",
        "determined", "disappointed", "disapproving", "disgusted", "distressed", "doubtful", "euphoric", "embarrassed",
        "disturbed", "enthusiastic", "entranced", "envious", "excited", "fearful", "grateful", "guilty", "horrified",
        "interested", "happy", "enamored", "nostalgic", "pained", "proud", "inspired", "relieved", "smitten", "sad",
        "satisfied", "desirous", "ashamed", "negatively surprised", "positively surprised", "sympathetic", "tired",
        "triumphant"
    ]

    @classmethod
    def scores_to_text(cls, emotion_scores: List[float]) -> str:
        if len(emotion_scores) == 48:
            adjectives = cls.ADJECTIVES_48
        elif len(emotion_scores) == 53:
            adjectives = cls.ADJECTIVES_53
        else:
            raise ValueError(f"Invalid length for emotion_scores {len(emotion_scores)}")

        # Return "neutral" if no emotions rate highly
        if all(emotion_score < cls.RANGES[0][0] for emotion_score in emotion_scores):
            return "neutral"

        # Construct phrases for all emotions that rate highly enough
        phrases = [""] * len(emotion_scores)
        for range_idx, (range_min, range_max) in enumerate(cls.RANGES):
            for emotion_idx, emotion_score in enumerate(emotion_scores):
                if range_min < emotion_score < range_max:
                    phrases[emotion_idx] = f"{cls.ADVERBS[range_idx]} {adjectives[emotion_idx]}"

        # Sort phrases by score
        sorted_indices = np.argsort(emotion_scores)[::-1]
        phrases = [phrases[i] for i in sorted_indices if phrases[i] != ""]

        # If there is only one phrase that rates highly, return it
        if len(phrases) == 0:
            return phrases[0]

        # Return all phrases separated by conjunctions
        return ", ".join(phrases[:-1]) + ", and " + phrases[-1]


In [ ]:
stringifier = Stringifier()
for emotion_embedding in emotion_embeddings:
    emotion_scores = [emotion["score"] for emotion in emotion_embedding]
    text = stringifier.scores_to_text(emotion_scores)
    print(text)

### Integrating with LLMs

This next part we leave up to you!

Take the text output of the last cell and integrate it into your LLM prompt.